In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder,OneHotEncoder
from sklearn import  preprocessing

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

/home/ali/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('ww-ii-data.csv')
df.head()

/tmp/ipykernel_187291/845397528.py:1: DtypeWarning: Columns (7,8,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ww-ii-data.csv')


,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,7/1/1942,1.016,NaN,25.555556,22.222222,23.888889,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,7/2/1942,0,NaN,28.888889,21.666667,25.555556,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,7/3/1942,2.54,NaN,26.111111,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,7/4/1942,2.54,NaN,26.666667,22.222222,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,7/5/1942,0,NaN,26.666667,21.666667,24.444444,0.0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119040 entries, 0 to 119039
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STA          119040 non-null  int64  
 1   Date         119040 non-null  object 
 2   Precip       119040 non-null  object 
 3   WindGustSpd  532 non-null     float64
 4   MaxTemp      119040 non-null  float64
 5   MinTemp      119040 non-null  float64
 6   MeanTemp     119040 non-null  float64
 7   Snowfall     117877 non-null  object 
 8   PoorWeather  34237 non-null   object 
 9   YR           119040 non-null  int64  
 10  MO           119040 non-null  int64  
 11  DA           119040 non-null  int64  
 12  PRCP         117108 non-null  object 
 13  DR           533 non-null     float64
 14  SPD          532 non-null     float64
 15  MAX          118566 non-null  float64
 16  MIN          118572 non-null  float64
 17  MEA          118542 non-null  float64
 18  SNF          117877 non-

In [4]:
df.isnull().sum()

STA                 0
Date                0
Precip              0
WindGustSpd    118508
MaxTemp             0
MinTemp             0
MeanTemp            0
Snowfall         1163
PoorWeather     84803
YR                  0
MO                  0
DA                  0
PRCP             1932
DR             118507
SPD            118508
MAX               474
MIN               468
MEA               498
SNF              1163
SND            113477
FT             119040
FB             119040
FTI            119040
ITH            119040
PGT            118515
TSHDSBRSGF      84803
SD3            119040
RHX            119040
RHN            119040
RVG            119040
WTE            119040
dtype: int64

In [5]:
X = df[['STA', 'YR', 'MO', 'DA']]
y = df['MaxTemp']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3243)

In [7]:
df.describe()

,STA,WindGustSpd,MaxTemp,MinTemp,MeanTemp,YR,MO,DA,DR,SPD,...,FT,FB,FTI,ITH,PGT,SD3,RHX,RHN,RVG,WTE
count,119040.000000,532.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,119040.000000,533.000000,532.000000,...,0.0,0.0,0.0,0.0,525.000000,0.0,0.0,0.0,0.0,0.0
mean,29659.435795,37.774534,27.045111,17.789511,22.411631,43.805284,6.726016,15.797530,26.998124,20.396617,...,NaN,NaN,NaN,NaN,12.085333,NaN,NaN,NaN,NaN,NaN
std,20953.209402,10.297808,8.717817,8.334572,8.297982,1.136718,3.425561,8.794541,15.221732,5.560371,...,NaN,NaN,NaN,NaN,5.731328,NaN,NaN,NaN,NaN,NaN
min,10001.000000,18.520000,-33.333333,-38.333333,-35.555556,40.000000,1.000000,1.000000,2.000000,10.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,11801.000000,29.632000,25.555556,15.000000,20.555556,43.000000,4.000000,8.000000,11.000000,16.000000,...,NaN,NaN,NaN,NaN,8.500000,NaN,NaN,NaN,NaN,NaN
50%,22508.000000,37.040000,29.444444,21.111111,25.555556,44.000000,7.000000,16.000000,32.000000,20.000000,...,NaN,NaN,NaN,NaN,11.600000,NaN,NaN,NaN,NaN,NaN
75%,33501.000000,43.059000,31.666667,23.333333,27.222222,45.000000,10.000000,23.000000,34.000000,23.250000,...,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN,NaN
max,82506.000000,75.932000,50.000000,34.444444,40.000000,45.000000,12.000000,31.000000,78.000000,41.000000,...,NaN,NaN,NaN,NaN,23.900000,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df.fillna(df.mean())

/tmp/ipykernel_187291/114435927.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [9]:
df.isnull().sum()

STA                 0
Date                0
Precip              0
WindGustSpd         0
MaxTemp             0
MinTemp             0
MeanTemp            0
Snowfall         1163
PoorWeather     84803
YR                  0
MO                  0
DA                  0
PRCP             1932
DR                  0
SPD                 0
MAX                 0
MIN                 0
MEA                 0
SNF              1163
SND                 0
FT             119040
FB             119040
FTI            119040
ITH            119040
PGT                 0
TSHDSBRSGF      84803
SD3            119040
RHX            119040
RHN            119040
RVG            119040
WTE            119040
dtype: int64

In [10]:
df2 = pd.read_csv('ww-ii-data.csv')
df2['Snowfall'].unique()

/tmp/ipykernel_187291/2719461289.py:1: DtypeWarning: Columns (7,8,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('ww-ii-data.csv')


array([0.0, nan, '0', '22.86', '33.02', '30.48', '2.54', '5.08', '10.16',
       '7.62', '20.32', '15.24', '17.78', '12.7', '25.4', '27.94',
       '43.18', '38.1', '45.72', '53.34', '58.42', '66.04', '76.2',
       '81.28', '73.66', '63.5', '50.8', '48.26', '60.96', '55.88',
       '35.56', '78.74', '40.64', '86.36', '83.82', '68.58', '#VALUE!',
       10.16, 7.62, 15.24, 2.54, 5.08, 12.7], dtype=object)

In [11]:
df2['Snowfall'] = pd.to_numeric(df2['Snowfall'], errors='coerce')

In [12]:
df = df.dropna(axis=1)

In [13]:
df.isnull().sum()

STA            0
Date           0
Precip         0
WindGustSpd    0
MaxTemp        0
MinTemp        0
MeanTemp       0
YR             0
MO             0
DA             0
DR             0
SPD            0
MAX            0
MIN            0
MEA            0
SND            0
PGT            0
dtype: int64

In [14]:
X = df[['STA', 'WindGustSpd', 'YR', 'MO', 'DA', 'SPD', 'MAX', 'MIN', 'MEA', 'SND', 'PGT', 'MeanTemp', 'MinTemp']]
y = df['MaxTemp']

In [15]:
oh=OneHotEncoder(drop='first',handle_unknown='ignore')
s1=StandardScaler()
m1=MinMaxScaler()

ct=ColumnTransformer(
                    [
#                         ('cat_encoder',oh,make_column_selector(dtype_include='object')),
                        ('StandardScaler',s1,make_column_selector(dtype_include='object')),
                        ('Numerical Scaler',m1,make_column_selector(dtype_exclude='object')),  
                    ]
                    ,remainder='passthrough')
X=ct.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3243)

In [17]:
lr = LinearRegression(n_jobs=-1)
svr = SVR(max_iter=2000)
dtr = DecisionTreeRegressor()
knnr = KNeighborsRegressor(n_neighbors=9,n_jobs=-1)
rfr = RandomForestRegressor(n_jobs=-1,n_estimators=100,oob_score=True)
abr = AdaBoostRegressor(n_estimators=500,learning_rate=.7)
sr =SGDRegressor()
cbr = CatBoostRegressor(iterations=2500, learning_rate=1,depth=16,eval_metric='MAE', verbose=150,task_type='GPU')
xg_reg  =xgb.XGBRFRegressor(colsample_bynode=.99,max_depth=15,min_child_weight= 11,alpha= 5.6353429991712695e-08,
    subsample= 0.99,
    colsample_bytree= 0.99,gamma= 1,
                            n_estimators=1000,n_jobs=-1,random_state=23,grow_policy= 'depthwise')

reg_list=[lr,svr,dtr,knnr,rfr,abr,sr,xg_reg]

In [18]:
%%time

counter=1
for reg in reg_list:
    reg.fit(X_train,y_train.values.ravel())
    print(f"Fitting {counter} done")
    counter+=1
#

Fitting 1 done


/home/ali/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Fitting 2 done
Fitting 3 done
Fitting 4 done
Fitting 5 done
Fitting 6 done
Fitting 7 done
Fitting 8 done
CPU times: user 17min 54s, sys: 14.5 s, total: 18min 8s
Wall time: 4min 27s


In [19]:
%%time
score_dict={}

for reg in reg_list:
    
    reg_name = reg.__class__.__name__
    
    pred=reg.predict(X_test)
    
    print("-"*50)
    
    print(reg_name)
    
    print(f"R2 Score : {r2_score(y_test,pred)}")
    print(f"Mean Absolute Error Score : {mean_absolute_error(y_test,pred)}")
    print(f"Mean Squared Error Score :{mean_squared_error(y_test,pred)}")
    
    
    score_dict[reg_name]={
        "R2 Score":r2_score(y_test,pred),
        "Mean Absolute Error Score":mean_absolute_error(y_test,pred),
        "Mean Squared Error Score" :mean_squared_error(y_test,pred),
    }

--------------------------------------------------
LinearRegression
R2 Score : 0.995537104506593
Mean Absolute Error Score : 0.040681876816649684
Mean Squared Error Score :0.32924948105775736
--------------------------------------------------
SVR
R2 Score : 0.9825351648078068
Mean Absolute Error Score : 0.9127918593060457
Mean Squared Error Score :1.2884657353692919
--------------------------------------------------
DecisionTreeRegressor
R2 Score : 0.9990494196173838
Mean Absolute Error Score : 0.002090800478048169
Mean Squared Error Score :0.07012893269458133
--------------------------------------------------
KNeighborsRegressor
R2 Score : 0.9784786155683399
Mean Absolute Error Score : 0.9082267192108237
Mean Squared Error Score :1.5877370792653884
--------------------------------------------------
RandomForestRegressor
R2 Score : 0.9988894540330346
Mean Absolute Error Score : 0.003319145758497362
Mean Squared Error Score :0.08193037095632534
------------------------------------------

In [20]:
score_pd=pd.DataFrame(score_dict).transpose().sort_values("R2 Score",ascending=False)
score_pd


,R2 Score,Mean Absolute Error Score,Mean Squared Error Score
DecisionTreeRegressor,0.999049,0.002091,0.070129
RandomForestRegressor,0.998889,0.003319,0.081930
XGBRFRegressor,0.997797,0.038819,0.162526
LinearRegression,0.995537,0.040682,0.329249
SGDRegressor,0.992527,0.275112,0.551292
SVR,0.982535,0.912792,1.288466
KNeighborsRegressor,0.978479,0.908227,1.587737
AdaBoostRegressor,0.946408,1.728356,3.953756
